<a href="https://colab.research.google.com/github/MuhammedIrshath49/ICT3113_G10/blob/main/Agent_CFO_Project_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agent CFO — Performance Optimization & Design

---
This is the starter notebook for your project. Follow the required structure below.


Colab integrates well with GitHub, allowing you to clone repositories, save notebooks, and share your work.

**1. Cloning a Repository**

You can clone a public or private GitHub repository directly into your Colab environment using the `!git clone` command. For private repositories, you'll need to provide credentials.

In [ ]:
# Clone a public repository
!git clone https://github.com/tensorflow/models.git

# For a private repository, you might need to use credentials
# !git clone https://<YOUR_GITHUB_USERNAME>:<YOUR_GITHUB_TOKEN>@github.com/<USERNAME>/<REPOSITORY>.git

**2. Saving a Notebook to GitHub**

You can save your Colab notebook directly to GitHub. Go to `File > Save a copy to GitHub`. You'll be prompted to authorize Colab to access your GitHub account and then you can select the repository and commit message.

**3. Loading a Notebook from GitHub**

To open a notebook from GitHub, go to `File > Open notebook`. In the dialog, select the "GitHub" tab and enter the GitHub URL or search for the repository and notebook.

**4. Authenticating with GitHub**

If you need to perform actions like pushing changes to a private repository, you'll need to authenticate. You can use a Personal Access Token (PAT) with the necessary permissions. Store your PAT securely (e.g., in Colab's Secrets Manager) and use it in your commands.

In [ ]:
# Example of configuring git with your name and email
!git config --global user.email "you@example.com"
!git config --global user.name "Your Name"

# Example of using a PAT for authentication (replace with your secret name)
# from google.colab import userdata
# github_token = userdata.get('github_pat')
# !git remote set-url origin https://<YOUR_GITHUB_USERNAME>:$github_token@github.com/<USERNAME>/<REPOSITORY>.git

These are the basic steps to get you started with using GitHub in Google Colab.

You will design and optimize an Agent CFO assistant for a listed company. The assistant should answer finance/operations questions using RAG (Retrieval-Augmented Generation) + agentic reasoning, with response time (latency) as the primary metric.

Your system must:
*   Ingest the company’s public filings.
*   Retrieve relevant passages efficiently.
*   Compute ratios/trends via tool calls (calculator, table parsing).
*   Produce answers with valid citations to the correct page/table.


## 1. Config & Secrets

Fill in your API keys in secrets. **Do not hardcode keys** in cells.

In [7]:
import os



COMPANY_NAME = "DBS Bank"
DATA_DIR = "/content/drive/MyDrive/ICT3113 Group 10/Data" # Define data directory here



### API Keys and Secrets Management

This project utilizes the Google Gemini API. To use it, you will need a Gemini API key.

**How to obtain a Gemini API Key:**
1.  Go to Google AI Studio ([https://aistudio.google.com/](https://aistudio.google.com/)).
2.  Create or select a project.
3.  Generate an API key.

**How to store your API Key securely in Colab Secrets:**
1.  In the left sidebar of your Colab notebook, click on the "🔑 Secrets" tab.
2.  Click on "Add new secret".
3.  For the **Name**, enter `GOOGLE_API_KEY`. This is the name the code will use to access the key.
4.  For the **Value**, paste your Gemini API key.
5.  Make sure the "Notebook access" toggle is turned ON for this secret.

You can then access the secret in your code using:

## 2. Data Download (Dropbox)

*   Annual Reports: last 3–5 years.
*   Quarterly Results Packs & MD&A (Management Discussion & Analysis).
*   Investor Presentations and Press Releases.
*   These files must be submitted later as a deliverable in the Dropbox data pack.
*   Upload them under `/content/data/`.

Scope limit: each team will ingest minimally 15 PDF files total.


## 3. System Requirements

**Retrieval & RAG**
*   Use a vector index (e.g., FAISS, LlamaIndex) + a keyword filter (BM25/ElasticSearch).
*   Citations must include: report name, year, page number, section/table.

**Agentic Reasoning**
*   Support at least 3 tool types: calculator, table extraction, multi-document compare.
*   Reasoning must follow a plan-then-act pattern (not a single unstructured call).

**Instrumentation**
*   Log timings for: T_ingest, T_retrieve, T_rerank, T_reason, T_generate, T_total.
*   Log: tokens used, cache hits, tools invoked.
*   Record p50/p95 latencies.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install -q sentence-transformers faiss-cpu numpy pandas scikit-learn PyMuPDF rank-bm25 google-generativeai tqdm

In [8]:
# TODO: Implement ingestion pipeline
import os
import time
import pickle
import logging
import pandas as pd
import numpy as np
import json
import re
from typing import List, Dict, Any, Tuple
from pathlib import Path

# RAG related libararies
import faiss
from sentence_transformers import SentenceTransformer
import fitz  # PyMuPDF for PDF processing
from rank_bm25 import BM25Okapi
import google.generativeai as genai # Gemini API for higher token limits

# Initialise logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# Tool for financial calculations
class CalculatorTool:
    def calculate_ratio(self, numerator: float, denominator: float, ratio_name: str = "") -> Dict[str, Any]:
        try:
            if denominator == 0:
                return {"error": f"Cannot calculate {ratio_name}: denominator is zero"}

            ratio = (numerator / denominator) * 100 if "ratio" in ratio_name.lower() else (numerator / denominator)
            return {
                "ratio_name": ratio_name,
                "numerator": numerator,
                "denominator": denominator,
                "result": round(ratio, 2),
                "formula": f"{numerator} / {denominator}"
            }
        except Exception as e:
            return {"error": str(e)}

    def trend_analysis(self, values: List[float], periods: List[str]) -> Dict[str, Any]:
        if len(values) != len(periods):
            return {"error": "Values and periods must have the same length"}

        if len(values) < 2:
            return {"error": "Need at least two data points for trend analysis"}

        # Calculate period-over-period changes
        changes = []
        for i in range(1, len(values)):
            if values[i-1] != 0:
                pct_change = ((values[i] - values[i-1]) / values[i-1]) * 100
                changes.append(round(pct_change, 2))
            else:
                changes.append(0)

        return {
            "periods": periods,
            "values": values,
            "period_changes": changes,
            "overall_trend": "increasing" if values[-1] > values[0] else "decreasing",
            "average_change": round(sum(changes) / len(changes), 2) if changes else 0
        }


# Tool for extracting table from dataset
class TableExtractionTool:
    def extract_financial_numbers(self, text: str) -> List[Dict[str, Any]]:
        # Pattern for numbers with currency/percentage
        patterns = [
            r'(\$|S\$|USD|SGD)?\s*(\d{1,3}(?:,\d{3})*(?:\.\d+)?)\s*(million|billion|thousand|m|bn|k)?',
            r'(\d{1,3}(?:,\d{3})*(?:\.\d+)?)\s*(%|percent|basis points|bps)',
            r'(NIM|CTI|ROE|ROA|CET1)\s*[:=]?\s*(\d+(?:\.\d+)?)\s*(%|bps)?'
        ]

        extracted = []
        for pattern in patterns:
            matches = re.finditer(pattern, text, re.IGNORECASE)
            for match in matches:
                extracted.append({
                    "text": match.group(0),
                    "value": match.group(2) if len(match.groups()) > 1 else match.group(1),
                    "context": text[max(0, match.start()-50):match.end()+50]  # 50 chars before and after
                })

        return extracted

    def parse_table_structure(self, text: str) -> Dict[str, Any]:
        lines = text.split('\n')
        table_lines = []

        for line in lines:
            # Look for lines that might be table rows (have multiple numbers/columns)
            if re.search(r'\d.*\d', line) and ('|' in line or '\t' in line or len(re.findall(r'\d+', line)) > 1):
                table_lines.append(line.strip())

        return {
            "potential_table_rows": table_lines[:10], # Return first 10 rows
            "row_count": len(table_lines)
        }


# Tool for comparing info across docs
class DocumentComparisonTool:
    def compare_metrics_across_docs(self, documents: List[Dict], metric_name: str) -> Dict[str, Any]:
        comparisons = []
        for doc in documents:
            # Extract metric from document text
            numbers = re.findall(r'\d+(?:\.\d+)?', doc.get('text', ''))
            filename = doc.get('metadata', {}).get('filename', 'unknown')

            comparisons.append({
                "document": filename,
                "metric_candidates": numbers[:5], # Return first 5 found numbers
                "text_snippet": doc.get('text', '')[:200] # First 200 chars
            })

        return {
            "metric_name": metric_name,
            "comparisons": comparisons
        }

# RAG functions
class CFORAGPipeline:
    def __init__(self, persist_dir="./cfo_rag_data"):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.persist_dir = persist_dir
        self.documents = []
        self.document_metadata = []
        self.index = None
        self.bm25 = None

        # Initialise tools
        self.calculator_tool = CalculatorTool()
        self.table_extraction_tool = TableExtractionTool()
        self.doc_comparison_tool = DocumentComparisonTool()

        # Create directory for persistence
        os.makedirs(self.persist_dir, exist_ok=True)

        # Performance tracking
        self.metrics = {
            'T_ingest': 0,
            'T_retrieve': 0,
            'T_rerank': 0,
            'documents_ingested': 0,
            }

        logger.info("Initialized CFO RAG Pipeline")

    def extract_text_from_pdf(self, pdf_path: str) -> List[Dict[str, Any]]:
        # for document chunking
        chunks = []

        try:
            doc = fitz.open(pdf_path)
            filename = Path(pdf_path).stem

            for page_num in range(len(doc)):
                page = doc[page_num]
                text = page.get_text()

                if text.strip():
                    # split by sentences/paragraphs for chunking
                    paragraphs = text.split('\n\n')

                    for i, paragraph in enumerate(paragraphs):
                        if len(paragraph.strip()) > 50:
                            chunk = {
                                'text': paragraph.strip(),
                                'metadata': {
                                    'filename': filename,
                                    'page': page_num + 1,
                                    'chunk_id': f"{filename}_p{page_num+1}_c{i+1}",
                                    'source_type': self._classify_document_type(filename)
                                }
                            }
                            chunks.append(chunk)

            doc.close()
            logger.info(f"Extracted {len(chunks)} text chunks from {pdf_path}")

        except Exception as e:
            logger.error(f"Error extracting text from {pdf_path}: {e}")

        return chunks

    def _classify_document_type(self, filename: str) -> str:
        # based on filename
        filename_lower = filename.lower()
        if 'annual' in filename_lower:
            return 'annual_report'
        elif any(q in filename_lower for q in ['1q', '2q', '3q', '4q', 'quarter']):
            return 'quarterly_report'
        elif 'performance' in filename_lower:
            return 'performance_summary'
        else:
            return 'financial_report'

        # document ingestion from data directory containing PDFs/datasets
    def ingest_documents(self, data_dir: str = "./content/data") -> Dict[str, Any]:
        # record time taken to ingest the documents
        start_time = time.time()

        pdf_files = list(Path(data_dir).glob("*.pdf"))
        if not pdf_files:
            raise ValueError(f"No PDF files found in {data_dir}")

        all_chunks = []

        # process each PDF file
        for pdf_file in pdf_files:
            chunks = self.extract_text_from_pdf(str(pdf_file))
            all_chunks.extend(chunks)

        # separate text and metadata
        texts = [chunk['text'] for chunk in all_chunks]
        metadatas = [chunk['metadata'] for chunk in all_chunks]

        self.documents = texts
        self.document_metadata = metadatas

        # Create embeddings
        embeddings = self.model.encode(texts, convert_to_numpy=True, show_progress_bar=True)

        # Create FAISS index
        dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatL2(dimension)
        self.index.add(embeddings.astype('float32'))

        # create BM25 index for keyword search
        tokenised_docs = [doc.lower().split() for doc in texts]
        self.bm25 = BM25Okapi(tokenised_docs)

        # save data
        self._save_data()

        # update metrics
        self.metrics['T_ingest'] = time.time() - start_time
        self.metrics['documents_ingested'] = len(texts)
        logger.info(f"Ingested {len(texts)} documents in {self.metrics['T_ingest']:.2f} seconds")

        return {
            'documents_processed': len(pdf_files),
            'chunks_created': len(texts),
            'ingestion_duration': self.metrics['T_ingest']
        }

    # retrieve relevant documents using hybrid search
    def hybrid_retrieve(self, query: str, top_k: int = 5) -> List[Dict[str, Any]]:
        if not self.documents or self.index is None:
            return []

        try:
            # --- Start retrieval timer ---
            start_retrieve = time.time()

            # Vector search
            query_embedding = self.model.encode([query], convert_to_numpy=True)
            vector_k = min(top_k * 2, len(self.documents))
            distances, indices = self.index.search(query_embedding.astype('float32'), vector_k)

            # BM25 keyword search
            bm25_scores = self.bm25.get_scores(query.lower().split())

            # Stop retrieval timer (only covers FAISS + BM25)
            self.metrics['T_retrieve'] = time.time() - start_retrieve

            # --- Start rerank timer ---
            start_rerank = time.perf_counter()

            combined_results = []
            for i, idx in enumerate(indices[0]):
                if idx < len(self.documents):
                    vector_score = 1 / (1 + distances[0][i])
                    bm25_score = bm25_scores[idx] if idx < len(bm25_scores) else 0
                    combined_score = vector_score + bm25_score

                    result = {
                        'text': self.documents[idx],
                        'metadata': self.document_metadata[idx],
                        'combined_score': combined_score,
                        'vector_score': vector_score,
                        'bm25_score': bm25_score,
                        'citation': f"{self.document_metadata[idx]['filename']}, Page {self.document_metadata[idx]['page']}"
                    }
                    combined_results.append(result)

            # Sorting and taking top_k
            combined_results.sort(key=lambda x: x['combined_score'], reverse=True)
            final_results = combined_results[:top_k]

            # Stop rerank timer (store in ms)
            self.metrics['T_rerank'] = (time.perf_counter() - start_rerank) * 1000

            return final_results

        except Exception as e:
            logger.error(f"Error during retrieval: {e}")
            return []


    def _save_data(self):
        # Save FAISS index
        try:
            with open(os.path.join(self.persist_dir, 'documents.pkl'), 'wb') as f:
                pickle.dump(self.documents, f)

            with open(os.path.join(self.persist_dir, 'metadata.pkl'), 'wb') as f:
                pickle.dump(self.document_metadata, f)

            if self.index is not None:
                faiss.write_index(self.index, os.path.join(self.persist_dir, 'faiss_index.bin'))

            if self.bm25 is not None:
                with open(os.path.join(self.persist_dir, 'bm25.pkl'), 'wb') as f:
                    pickle.dump(self.bm25, f)

            logger.info("Saved ingestion data to disk")

        except Exception as e:
            logger.error(f"Error saving data: {e}")

cfo_rag = CFORAGPipeline()

# Ingest documents from data directory
print("=== Starting document ingestion ===")
ingestion_result = cfo_rag.ingest_documents(data_dir=DATA_DIR) # Use the variable here
print(f"Processed: {ingestion_result['documents_processed']} PDFs")
print(f"Created: {ingestion_result['chunks_created']} text chunks")
print(f"Ingestion Time: {ingestion_result['ingestion_duration']:.2f} seconds")

# Test retrieval
test_query = "Net Interest Margin trend over the past 3 years"
retrieved_docs = cfo_rag.hybrid_retrieve(test_query, top_k=3)

print(f"\n=== Retrieval Test ===")
print(f"Query: {test_query}")
print(f"Retrieved {len(retrieved_docs)} documents:")


if retrieved_docs:
    for i, doc in enumerate(retrieved_docs, 1):
        print(f"\nDocument {i}: {doc['citation']}")
        print(f"Combined Score: {doc['combined_score']:.4f}")
        print(f"Text Preview: {doc['text'][:150].replace(chr(10), ' ')}...")  # Print first 150 chars
else:
    print("No documents retrieved.")

=== Starting document ingestion ===


Batches:   0%|          | 0/31 [00:00<?, ?it/s]

Processed: 24 PDFs
Created: 965 text chunks
Ingestion Time: 89.87 seconds

=== Retrieval Test ===
Query: Net Interest Margin trend over the past 3 years
Retrieved 3 documents:

Document 1: DBS Annual Report 2023, Page 13
Combined Score: 11.4779
Text Preview: 20 DBS ANNUAL REPORT 2023       BUILDING A SUSTAINABLE ADVANTAGE CFO statement We achieved a record performance for the third consecutive year with  t...

Document 2: 2Q22_performance_summary, Page 7
Combined Score: 11.2330
Text Preview: DBS GROUP HOLDINGS LTD AND ITS SUBSIDIARIES    5    First Half    First-half net profit was $3.62 billion, 3% below the  previous year’s record. Busin...

Document 3: 1Q22_CFO_presentation, Page 15
Combined Score: 8.2890
Text Preview: In summary – strong first-quarter operating performance 15 Strong first quarter as business momentum healthy and growth broad-based,  expenses well-ma...


## 4. Baseline Pipeline

**Baseline (starting point)**
*   Naive chunking.
*   Single-pass vector search.
*   One LLM call, no caching.

In [12]:
# TODO: Implement baseline retrieval + generation
# =============================
# Part 4. Baseline Pipeline
# =============================
import os
import google.generativeai as genai
from google.colab import userdata

# Configure Gemini using Colab secrets
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# Load model
llm_model = genai.GenerativeModel("gemini-2.5-flash")


def baseline_pipeline(query: str, top_k: int = 5):
    """
    Runs naive RAG pipeline: retrieval + single LLM call.
    """
    # Retrieve relevant docs
    retrieved_docs = cfo_rag.hybrid_retrieve(query, top_k=top_k)
    if not retrieved_docs:
        return {"error": "No documents retrieved."}

    # Build context
    context = "\n\n".join([f"{doc['citation']}: {doc['text']}" for doc in retrieved_docs])

    # Prompt
    prompt = f"""
You are a financial analyst assistant.
Answer the user query based only on the provided reports.
Include citations (filename + page).

Query: {query}

Reports:
{context}
"""
    # Call LLM
    response = llm_model.generate_content(prompt)

    return {
        "query": query,
        "citations": [doc["citation"] for doc in retrieved_docs],
        "raw_docs": [doc["text"][:300] for doc in retrieved_docs],  # preview only
        "answer": response.text.strip()
    }

# 🔹 Example run
result = baseline_pipeline("Net Interest Margin trend over the past 3 years", top_k=3)
print("=== Baseline Answer ===")
print(result["answer"])
print("\nCitations:", result["citations"])

=== Baseline Answer ===
Over the past three years, DBS's Net Interest Margin (NIM) has shown a trend of decline followed by a significant increase:

*   **2023:** The full-year net interest margin expanded by 40 basis points to **2.15%**. Quarterly net interest margin continued to rise in the first three quarters before easing in the fourth quarter (DBS Annual Report 2023, Page 13).
*   **2022:** The full-year NIM saw an increase of 62 basis points (DBS Annual Report 2023, Page 13). The NIM started to rise in the first quarter with the commencement of interest rate hikes, and this improvement accelerated in the second quarter. The first half NIM was **1.52%** (2Q22_performance_summary, Page 7). This rise followed three years of decline (2Q22_performance_summary, Page 7; 1Q22_CFO_presentation, Page 15).
*   **2021:** Net interest margin had been declining since 2019 (2Q22_performance_summary, Page 7; 1Q22_CFO_presentation, Page 15). The first half NIM for 2021 was approximately 1.47% (c

## 5. Benchmark Runner

Run these 3 standardized queries. Produce JSON then prose answers with citations. These are the standardized queries.

*   Net Interest Margin (NIM) trend over last 5 quarters, values and 1–2 lines of explanation.
    *   Expected: quarterly financial highlights.
*   Operating Expenses (Opex) YoY for last 3 years; top 3 drivers from MD&A.
    *   Expected: Opex table + MD&A commentary.
*   Cost-to-Income Ratio (CTI) for last 3 years; show working + implications.
    *   Expected: Operating Income & Opex lines.


In [16]:
# TODO: Implement benchmark runner
# =============================
# Part 5. Benchmark Runner
# =============================

benchmark_queries = [
    {
        "name": "NIM Trend",
        "query": "Net Interest Margin (NIM) trend over last 5 quarters, values and 1–2 lines of explanation.",
        "expected": "Quarterly financial highlights"
    },
    {
        "name": "Opex YoY",
        "query": "Operating Expenses (Opex) YoY for last 3 years; top 3 drivers from MD&A.",
        "expected": "Opex table + MD&A commentary"
    },
    {
        "name": "Cost-to-Income Ratio",
        "query": "Cost-to-Income Ratio (CTI) for last 3 years; show working + implications.",
        "expected": "Operating Income & Opex lines"
    }
]

# =============================
# Benchmark Runner with Instrumentation
# =============================

def run_benchmark_instrumented(queries, top_k=5):
    results = []
    for q in queries:
        print(f"\n=== Running Benchmark: {q['name']} ===")
        output = instrumented_pipeline(q["query"], top_k=top_k)
        results.append({
            "name": q["name"],
            "query": q["query"],
            "expected": q["expected"],
            "citations": output.get("citations", []),
            "answer": output.get("answer", "Error: no answer")
        })
    return results

# 🔹 Run the benchmarks
benchmark_results = run_benchmark_instrumented(benchmark_queries, top_k=5)

# Show answers
for res in benchmark_results:
    print(f"\n=== {res['name']} ===")
    print(f"Query: {res['query']}")
    print(f"Answer: {res['answer']}\n")
    print(f"Citations: {res['citations']}")



=== Running Benchmark: NIM Trend ===


/tmp/ipython-input-3459150787.py:81: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  logs = pd.concat([logs, pd.DataFrame([new_row])], ignore_index=True)



=== Timing Summary for Query: Net Interest Margin (NIM) trend over last 5 quarters, values and 1–2 lines of explanation. ===
| Stage    | Time        |
|----------|-------------|
| Retrieve | 0.0410 sec  |
| Rerank   | 0.126 ms    |
| Reason   | 14.5394 sec |
| Generate | 5.1177 sec  |
| Total    | 19.6982 sec |

=== Running Benchmark: Opex YoY ===

=== Timing Summary for Query: Operating Expenses (Opex) YoY for last 3 years; top 3 drivers from MD&A. ===
| Stage    | Time        |
|----------|-------------|
| Retrieve | 0.0381 sec  |
| Rerank   | 0.128 ms    |
| Reason   | 43.3552 sec |
| Generate | 8.1705 sec  |
| Total    | 51.5640 sec |

=== Running Benchmark: Cost-to-Income Ratio ===

=== Timing Summary for Query: Cost-to-Income Ratio (CTI) for last 3 years; show working + implications. ===
| Stage    | Time        |
|----------|-------------|
| Retrieve | 0.0299 sec  |
| Rerank   | 0.097 ms    |
| Reason   | 40.8283 sec |
| Generate | 6.0280 sec  |
| Total    | 46.8863 sec |

===

In [17]:
# Define instrumented pipeline and benchmark runner functions
import pandas as pd
import time
from tabulate import tabulate

# Make sure logs DataFrame is accessible (if not defined globally elsewhere)
try:
    logs
except NameError:
    logs = pd.DataFrame(columns=[
        'Query',
        'T_ingest (sec)', 'T_retrieve (sec)', 'T_rerank (ms)', 'T_reason (sec)',
        'T_generate (sec)', 'T_total (sec)',
        'Tokens', 'CacheHits', 'Tools'
    ])


def instrumented_pipeline(query: str, top_k: int = 5):
    """
    Runs pipeline with instrumentation (timing + usage logging).
    """
    global logs
    timings = {}
    start_total = time.time()

    # --- Retrieval ---
    retrieved_docs = cfo_rag.hybrid_retrieve(query, top_k=top_k)
    timings['T_retrieve'] = cfo_rag.metrics.get('T_retrieve', 0)
    timings['T_rerank']  = cfo_rag.metrics.get('T_rerank', 0)  # already in ms


    if not retrieved_docs:
        return {"error": "No documents retrieved."}

    # --- Build context ---
    context = "\n\n".join([f"{doc['citation']}: {doc['text']}" for doc in retrieved_docs])

    # --- Reasoning step ---
    start_reason = time.time()
    reasoning_prompt = f"""
    Summarize the key financial figures and trends in a structured way.
    Do not generate the final answer yet.

    Query: {query}
    Reports:
    {context}
    """
    reasoning_output = llm_model.generate_content(reasoning_prompt)
    timings['T_reason'] = time.time() - start_reason

    # --- Generation step ---
    start_generate = time.time()
    final_prompt = f"""
    You are a financial analyst assistant.
    Based on the reasoning and reports, provide the final answer with citations.

    Reasoning:
    {reasoning_output.text}

    Query: {query}
    """
    response = llm_model.generate_content(final_prompt)
    timings['T_generate'] = time.time() - start_generate

    # --- Total ---
    timings['T_total'] = time.time() - start_total

    # --- Log row ---
    new_row = {
        'Query': query,
        'T_ingest (sec)': cfo_rag.metrics.get('T_ingest', 0),
        'T_retrieve (sec)': timings['T_retrieve'],
        'T_rerank (ms)': timings['T_rerank'],
        'T_reason (sec)': timings['T_reason'],
        'T_generate (sec)': timings['T_generate'],
        'T_total (sec)': timings['T_total'],
        'Tokens': None, # Placeholder
        'CacheHits': 0, # Placeholder
        'Tools': ['Retriever', 'Reranker', 'LLM'] # Placeholder
    }
    logs = pd.concat([logs, pd.DataFrame([new_row])], ignore_index=True)

    # --- Print per-query summary ---
    summary = [
        ["Retrieve", f"{timings['T_retrieve']:.4f} sec"],
        ["Rerank",   f"{timings['T_rerank']:.3f} ms"],
        ["Reason",   f"{timings['T_reason']:.4f} sec"],
        ["Generate", f"{timings['T_generate']:.4f} sec"],
        ["Total",    f"{timings['T_total']:.4f} sec"],
    ]
    print(f"\n=== Timing Summary for Query: {query} ===")
    print(tabulate(summary, headers=["Stage", "Time"], tablefmt="github"))


    return {
        "query": query,
        "citations": [doc["citation"] for doc in retrieved_docs],
        "reasoning": reasoning_output.text.strip(),
        "answer": response.text.strip()
    }


def run_benchmark_instrumented(queries, top_k=5):
    results = []
    for q in queries:
        print(f"\n=== Running Instrumented Benchmark: {q['name']} ===")
        output = instrumented_pipeline(q["query"], top_k=top_k)
        results.append({
            "name": q["name"],
            "query": q["query"],
            "expected": q["expected"],
            "citations": output.get("citations", []),
            "reasoning": output.get("reasoning", "N/A"),
            "answer": output.get("answer", "Error: no answer")
        })
    return results

## 6. Instrumentation

Log timings: T_ingest, T_retrieve, T_rerank, T_reason, T_generate, T_total. Log tokens, cache hits, tools.

In [18]:
# =============================
# Part 6. Instrumentation (Final Version)
# =============================

import pandas as pd
import time
from tabulate import tabulate  # pretty summary tables

# Create logs DataFrame
logs = pd.DataFrame(columns=[
    'Query',
    'T_ingest (sec)', 'T_retrieve (sec)', 'T_rerank (ms)', 'T_reason (sec)',
    'T_generate (sec)', 'T_total (sec)',
    'Tokens', 'CacheHits', 'Tools'
])

# 🔹 Run instrumented benchmarks
benchmark_results_instrumented = run_benchmark_instrumented(benchmark_queries, top_k=5)

# 🔹 Show instrumentation logs (DataFrame)
logs


=== Running Instrumented Benchmark: NIM Trend ===


/tmp/ipython-input-3459150787.py:81: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  logs = pd.concat([logs, pd.DataFrame([new_row])], ignore_index=True)



=== Timing Summary for Query: Net Interest Margin (NIM) trend over last 5 quarters, values and 1–2 lines of explanation. ===
| Stage    | Time        |
|----------|-------------|
| Retrieve | 0.0397 sec  |
| Rerank   | 0.152 ms    |
| Reason   | 31.6278 sec |
| Generate | 3.2250 sec  |
| Total    | 34.8927 sec |

=== Running Instrumented Benchmark: Opex YoY ===

=== Timing Summary for Query: Operating Expenses (Opex) YoY for last 3 years; top 3 drivers from MD&A. ===
| Stage    | Time        |
|----------|-------------|
| Retrieve | 0.0405 sec  |
| Rerank   | 0.148 ms    |
| Reason   | 22.8699 sec |
| Generate | 5.3159 sec  |
| Total    | 28.2265 sec |

=== Running Instrumented Benchmark: Cost-to-Income Ratio ===

=== Timing Summary for Query: Cost-to-Income Ratio (CTI) for last 3 years; show working + implications. ===
| Stage    | Time        |
|----------|-------------|
| Retrieve | 0.0242 sec  |
| Rerank   | 0.091 ms    |
| Reason   | 41.7820 sec |
| Generate | 5.9564 sec  |
| Tot

,Query,T_ingest (sec),T_retrieve (sec),T_rerank (ms),T_reason (sec),T_generate (sec),T_total (sec),Tokens,CacheHits,Tools
0,Net Interest Margin (NIM) trend over last 5 qu...,89.87236,0.039718,0.15230,31.627774,3.224990,34.892684,None,0,"[Retriever, Reranker, LLM]"
1,Operating Expenses (Opex) YoY for last 3 years...,89.87236,0.040478,0.14800,22.869945,5.315903,28.226525,None,0,"[Retriever, Reranker, LLM]"
2,Cost-to-Income Ratio (CTI) for last 3 years; s...,89.87236,0.024154,0.09091,41.781966,5.956351,47.762585,None,0,"[Retriever, Reranker, LLM]"


### Running Baseline Pipeline (Part 4)

In [19]:
# Run the baseline pipeline example
result = baseline_pipeline("Net Interest Margin trend over the past 3 years", top_k=3)
print("=== Baseline Answer ===")
print(result["answer"])
print("\nCitations:", result["citations"])

=== Baseline Answer ===
The Net Interest Margin (NIM) trend over the past 3 years for DBS is as follows:

*   **2021:** The Net Interest Margin had been declining since 2019. For the first half of 2021, the NIM was 1.47% (derived from being five basis points lower than 1H 2022 NIM of 1.52%). (2Q22_performance_summary, Page 7; 1Q22_CFO_presentation, Page 15)
*   **2022:** The NIM began to rise in the first quarter of 2022, accelerating in the second quarter due to interest rate hikes. The first-half NIM was 1.52%. The quarterly NIM saw an increase of 62 basis points across the four quarters of 2022. The full-year NIM for 2022 can be derived as 1.75% (2.15% for 2023 minus the 40 basis points expansion). (2Q22_performance_summary, Page 7; 1Q22_CFO_presentation, Page 15; DBS Annual Report 2023, Page 13)
*   **2023:** The full-year NIM expanded by 40 basis points to reach 2.15%. Quarterly NIM continued to rise in the first three quarters before easing in the fourth quarter. (DBS Annual Repo

### Running Instrumented Benchmark (Part 5)

In [20]:
# Run the instrumented benchmarks
benchmark_results_instrumented = run_benchmark_instrumented(benchmark_queries, top_k=5)

# Show instrumentation logs (DataFrame)
display(logs)


=== Running Instrumented Benchmark: NIM Trend ===

=== Timing Summary for Query: Net Interest Margin (NIM) trend over last 5 quarters, values and 1–2 lines of explanation. ===
| Stage    | Time        |
|----------|-------------|
| Retrieve | 0.0292 sec  |
| Rerank   | 0.079 ms    |
| Reason   | 9.4137 sec  |
| Generate | 3.9832 sec  |
| Total    | 13.4263 sec |

=== Running Instrumented Benchmark: Opex YoY ===

=== Timing Summary for Query: Operating Expenses (Opex) YoY for last 3 years; top 3 drivers from MD&A. ===
| Stage    | Time        |
|----------|-------------|
| Retrieve | 0.0276 sec  |
| Rerank   | 0.082 ms    |
| Reason   | 33.5370 sec |
| Generate | 7.7522 sec  |
| Total    | 41.3169 sec |

=== Running Instrumented Benchmark: Cost-to-Income Ratio ===


ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2858.08ms



=== Timing Summary for Query: Cost-to-Income Ratio (CTI) for last 3 years; show working + implications. ===
| Stage    | Time        |
|----------|-------------|
| Retrieve | 0.0343 sec  |
| Rerank   | 0.136 ms    |
| Reason   | 44.9376 sec |
| Generate | 13.8626 sec |
| Total    | 58.8346 sec |


,Query,T_ingest (sec),T_retrieve (sec),T_rerank (ms),T_reason (sec),T_generate (sec),T_total (sec),Tokens,CacheHits,Tools
0,Net Interest Margin (NIM) trend over last 5 qu...,89.87236,0.039718,0.15230,31.627774,3.224990,34.892684,None,0,"[Retriever, Reranker, LLM]"
1,Operating Expenses (Opex) YoY for last 3 years...,89.87236,0.040478,0.14800,22.869945,5.315903,28.226525,None,0,"[Retriever, Reranker, LLM]"
2,Cost-to-Income Ratio (CTI) for last 3 years; s...,89.87236,0.024154,0.09091,41.781966,5.956351,47.762585,None,0,"[Retriever, Reranker, LLM]"
3,Net Interest Margin (NIM) trend over last 5 qu...,89.87236,0.029218,0.07886,9.413718,3.983233,13.426270,None,0,"[Retriever, Reranker, LLM]"
4,Operating Expenses (Opex) YoY for last 3 years...,89.87236,0.027552,0.08180,33.537036,7.752197,41.316890,None,0,"[Retriever, Reranker, LLM]"
5,Cost-to-Income Ratio (CTI) for last 3 years; s...,89.87236,0.034264,0.13602,44.937578,13.862587,58.834595,None,0,"[Retriever, Reranker, LLM]"


## 7. Optimizations

**Required Optimizations**

Each team must implement at least:
*   2 retrieval optimizations (e.g., hybrid BM25+vector, smaller embeddings, dynamic k).
*   1 caching optimization (query cache or ratio cache).
*   1 agentic optimization (plan pruning, parallel sub-queries).
*   1 system optimization (async I/O, batch embedding, memory-mapped vectors).

In [ ]:
# TODO: Implement optimizations


## 8. Results & Plots

Show baseline vs optimized. Include latency plots (p50/p95) and accuracy tables.

In [ ]:
# TODO: Generate plots with matplotlib


testing